In [1]:
# imports for conputer vision API (OpenCV, Mediapipe)
import cv2
import mediapipe as mp
import time

# imports for Spotipy API
import spotipy
import json
import webbrowser
import pandas as pd
import spotipy.util as util

In [2]:
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [3]:
import os
os.sys.path
# checking if I am using the right version of Python
# if it's 310 - it's good

['C:\\Users\\Eunah Lee\\Classes\\Intro To Creative Code\\TheConducter',
 'C:\\Users\\Eunah Lee\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip',
 'C:\\Users\\Eunah Lee\\AppData\\Local\\Programs\\Python\\Python310\\DLLs',
 'C:\\Users\\Eunah Lee\\AppData\\Local\\Programs\\Python\\Python310\\lib',
 'C:\\Users\\Eunah Lee\\AppData\\Local\\Programs\\Python\\Python310',
 '',
 'C:\\Users\\Eunah Lee\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages',
 'C:\\Users\\Eunah Lee\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32',
 'C:\\Users\\Eunah Lee\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Eunah Lee\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\Pythonwin']

In [4]:
# open file with keys and set the path to your credentials JSON file
credentials = "gitignore/spotify_keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [5]:
# reading the keys and assign each to a variable
# A redirect URI, or reply URL, is the location where the authorization server sends the user once the app has been 
# successfully authorized and granted an authorization code or access token.
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]

In [6]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-read-currently-playing user-modify-playback-state app-remote-control streaming'
# streaming - might need this but this needs premium account
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)
# this creates a cache file in the folder that saves the token
# you also have to .gitignore this

In [7]:
token
# make sure to clear the output on this cell

'BQC5PsvtR7-uPWC2fVCka47G5LHGPBzJaWsQPrGZwnBOH5OCxZK3Z5C5NlG9WlaIbWo_vhvZHPCZdFGTWlUTWHIpixl9y0hmJWP_tvFXFuVzJEluq8kjLhR24IPApZ6KXaZcA1tt0glokHMtjs-YqFosZ-UecWfbV6FD78Ipm03TzAhbLoLsr2gqsRCOD78N-FFyjln0PUQAmCHdAYSOckn29OtSzFJtgLspaEV5PgsC0HJmskyTqZ_hSVcBhsdB'

In [8]:
# Create Spotify Object
sp = spotipy.Spotify(auth=token)

In [9]:
sp.devices() == 0

False

In [10]:
# Before running this code, make sure to open spotify on your device browser, in order to get detected
# Also make sure to refresh it if you are already there

# Make sure to play a song in your browser to refresh

devices = sp.devices()
print(devices)

{'devices': [{'id': '9107cb864677026d874cb5a6b1cca15286142742', 'is_active': False, 'is_private_session': False, 'is_restricted': False, 'name': 'Mobile Web Player', 'supports_volume': True, 'type': 'Smartphone', 'volume_percent': 10}, {'id': 'f560121bd5782fb50a8aa147fd6573bcf04182e5', 'is_active': True, 'is_private_session': False, 'is_restricted': False, 'name': 'Web Player (Chrome)', 'supports_volume': True, 'type': 'Computer', 'volume_percent': 10}]}


In [11]:
# Assigning device id into var
deviceID_01 = devices['devices'][-1]['id']
deviceID_02 = devices['devices'][-2]['id']

print(deviceID_01)
print(deviceID_02)

f560121bd5782fb50a8aa147fd6573bcf04182e5
9107cb864677026d874cb5a6b1cca15286142742


In [12]:
song_play = sp.current_playback() #get playback status
print(song_play)

{'device': {'id': 'f560121bd5782fb50a8aa147fd6573bcf04182e5', 'is_active': True, 'is_private_session': False, 'is_restricted': False, 'name': 'Web Player (Chrome)', 'supports_volume': True, 'type': 'Computer', 'volume_percent': 10}, 'shuffle_state': False, 'repeat_state': 'off', 'timestamp': 1701195545140, 'context': {'external_urls': {'spotify': 'https://open.spotify.com/album/23fwKUYyu1kvRnDR2imBnq'}, 'href': 'https://api.spotify.com/v1/albums/23fwKUYyu1kvRnDR2imBnq', 'type': 'album', 'uri': 'spotify:album:23fwKUYyu1kvRnDR2imBnq'}, 'progress_ms': 31261, 'item': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/30AghIfoMJrMo1YqoUkEMM'}, 'href': 'https://api.spotify.com/v1/artists/30AghIfoMJrMo1YqoUkEMM', 'id': '30AghIfoMJrMo1YqoUkEMM', 'name': 'Huh', 'type': 'artist', 'uri': 'spotify:artist:30AghIfoMJrMo1YqoUkEMM'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO'

In [13]:
if_playing = song_play["is_playing"] #get playback status
print(if_playing)

True


In [14]:
# Resume playback
sp.start_playback()

In [15]:
# stop playback
sp.pause_playback()

In [16]:
sp.transfer_playback(deviceID_01)

In [17]:
sp.transfer_playback(deviceID_02)

In [18]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

cap = cv2.VideoCapture(0)

# cap = cv2.flip(src, 0)
# # flipping the video

spotify_url = 'https://open.spotify.com/'

on_left = False
last_on_left_time = time.time()

In [19]:
#Chcking the video size
video_width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
video_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`
print(video_width, video_height)

640.0 480.0


In [ ]:
while True:
    ret, frame = cap.read()
    # ret is a boolean variable that returns true if the frame is available.
    # frame is an image array vector captured based on the default frames per second defined explicitly or implicitly

    if not ret: 
        break
        # if there is no frame avaiable, escape from the loop. if it says continue, continue with the loop

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
    # changing the default BRG colour to RGB

    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:

            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            # tracking index finger

            if index_tip.x < 0.5 and on_left == False: 
            # see if the x position of the index finger is on the left side of the camera
            # x position value is 0-1
            # the second part checks if your hand is already on the left side
                
                sp.transfer_playback(deviceID_01)
                on_left = True
                # change on_left value to be True
            
            elif index_tip.x >= 0.5 and on_left == True:
                sp.transfer_playback(deviceID_02)
                on_left = False
                sp.volume(int(index_tip.y* 100))

            if index_tip.y > 0.1:
            sp.volume(int(index_tip.y * 100))
            # volume must be and int
            

            # drawing a circle on each landmark(tracking points)
            for landmark in hand_landmarks.landmark:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
                # cv2.circle(image, center_coordinates, radius, color, thickness)

    # cv2.putText(frame, f'Touches: {touch_count}/3', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [55]:
sp.volume(100)

In [22]:
###############################################
# GOAL2 -> map the y value to adjust the volume
###############################################

while True:
    ret, frame = cap.read()
    # ret is a boolean variable that returns true if the frame is available.
    # frame is an image array vector captured based on the default frames per second defined explicitly or implicitly

    if not ret: 
        break
        # if there is no frame avaiable, escape from the loop. if it says continue, continue with the loop

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
    # changing the default BRG colour to RGB

    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:

            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            # tracking index finger

            if index_tip.y < 0.1:
                sp.volume(100)
            
            elif index_tip.y >= 0.1 and index_tip.y < 0.2:
                sp.volume(90)

            elif index_tip.y >= 0.2 and index_tip.y < 0.3:
                sp.volume(80)

            elif index_tip.y >= 0.3 and index_tip.y < 0.4:
                sp.volume(70)

            elif index_tip.y >= 0.4 and index_tip.y < 0.5:
                sp.volume(60)

            elif index_tip.y >= 0.5 and index_tip.y < 0.6:
                sp.volume(50)

            elif index_tip.y >= 0.6 and index_tip.y < 0.7:
                sp.volume(40)

            elif index_tip.y >= 0.7 and index_tip.y < 0.8:
                sp.volume(30)

            elif index_tip.y >= 0.8 and index_tip.y < 0.9:
                sp.volume(20)
                            
            elif index_tip.y >= 0.9 and index_tip.y < 1:
                sp.volume(10)

            # volume must be and int
            
            # drawing a circle on each landmark(tracking points)
            for landmark in hand_landmarks.landmark:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
                # cv2.circle(image, center_coordinates, radius, color, thickness)

    # cv2.putText(frame, f'Touches: {touch_count}/3', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

HTTP Error for PUT to https://api.spotify.com/v1/me/player/volume?volume_percent=30 with Params: {} returned 403 due to Player command failed: Restriction violated


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/me/player/volume?volume_percent=30:
 Player command failed: Restriction violated, reason: UNKNOWN